In [1]:
# Dependencies
import requests
import json
import pandas as pd

from sqlalchemy import create_engine
# Google developer API key
from config import gkey

In [2]:
accidents_csv = "../../Resources/raw/Saferparks-dataset-legacy-v2.csv"
table_name = "accident_data"

geocode_json_base = "https://maps.googleapis.com/maps/api/geocode/json?"

In [103]:
df = pd.read_csv(accidents_csv, encoding="ANSI")

In [59]:
unique_cities = df["acc_city"].unique()

city_state_df = df[["acc_state", "acc_city"]].loc[df["acc_city"].isin(unique_cities)]
city_state_df.drop_duplicates(inplace=True)
city_state_df.reset_index(inplace=True, drop=True)
city_state_df.head()

,acc_state,acc_city
0,WI,Green Bay
1,WI,Madison
2,WI,Wisconsin Dells
3,WI,Phillips
4,WI,West Allis


In [87]:
city_state_df.loc[city_state_df["acc_city"]=="Fairfield"]


,acc_state,acc_city
238,IL,Fairfield
356,NJ,Fairfield


In [65]:

city_lat = []
city_lng = []
for i in range(len(city_state_df)):
    params = {
        "address": f'{city_state_df.loc[i,"acc_city"]}, {city_state_df.loc[i,"acc_state"]}',
        "key": gkey
    }
    geo_data = requests.get(geocode_json_base, params).json()
    try:
        city_lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
        city_lng.append(geo_data["results"][0]["geometry"]["location"]["lng"])
        print(f'{city_state_df.loc[i,"acc_city"]}, {city_state_df.loc[i,"acc_state"]}')
    except:
        print(f'Address: {city_state_df.loc[i,"acc_city"]}, {city_state_df.loc[i,"acc_state"]} not found')
        city_lat.append("null")
        city_lng.append("null") 


Green Bay, WI
Madison, WI
Wisconsin Dells, WI
Phillips, WI
West Allis, WI
Milwaukee, WI
nan, WI
Twin Lakes, WI
Oshkosh, WI
Black River Falls, WI
Lakewood, WI
Portage, WI
Baraboo, WI
Lake Delton, WI
Marshall, WI
Shullsburg, WI
EauClaire, WI
Waterloo, WI
Hartford, WI
Madison, WV
Elkins, WV
Fairmont, WV
Lewisburg, WV
Reedsville, WV
Ripley, WV
Matewan, WV
Hurricane, WV
Bell, WV
Wisconsin Dells, OH
Aurora, OH
nan, OH
Mason, OH
Union City, OH
Sandusky, OH
Latham, OH
Kings Island, OH
Hillsboro, OH
Columbus, OH
Puyallup, WA
nan, WA
Wenatchee, WA
Seattle, WA
Redmond, WA
Spokane, WA
Tukwila, WA
Anaheim, CA
Buena Park, CA
Los Angeles, CA
Riverside, CA
Upland, CA
San Dimas, CA
San Diego, CA
Irvine, CA
Palm Springs, CA
Lake Gregory, CA
Escondido, CA
San Jose, CA
Carlsbad, CA
Vista, CA
Santa Barbara, CA
Angels Camp, CA
Universal City, CA
Santa Monica, CA
nan, CA
Santa Clara, CA
Vallejo, CA
Gilroy, CA
Pleasanton, CA
Sacramento, CA
Roseville, CA
Pomona, CA
Clovis, CA
Concord, CA
Santa Cruz, CA
Arroyo 

In [101]:
city_df = pd.DataFrame({
    "acc_city": city_state_df["acc_city"],
    "city_lat": city_lat,
    "city_lng": city_lng
})
city_df = city_df[city_df.acc_city != "null"]
city_df.dropna(inplace=True)
city_df.head()

,acc_city,city_lat,city_lng
0,Green Bay,44.5133,-88.0133
1,Madison,43.0731,-89.4012
2,Wisconsin Dells,43.6275,-89.771
3,Phillips,45.6966,-90.4004
4,West Allis,43.0167,-88.007


In [102]:
len(city_df)

528

In [70]:
acc_states = df["acc_state"].unique()
state_lat = []
state_lng = []
for state in acc_states:
    params = {
        "address": state,
        "key": gkey
    }
    geo_data = requests.get(geocode_json_base, params).json()
    try:
        state_lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
        state_lng.append(geo_data["results"][0]["geometry"]["location"]["lng"])
        print(state)
    except:
        print(f'address: {city} not found')
        state_lat.append("null")
        state_lng.append("null") 

WI
WV
OH
WA
CA
FL
CO
IL
KY
OK
TX
PA
NY
NJ
NV
MA
MO
MD
MI
NC
AR
GA
IA
VA
TN
AZ
CT
IN
RI
LA


In [71]:
state_df = pd.DataFrame({
    "acc_state": acc_states,
    "state_lat": state_lat,
    "state_lng": state_lng
})

state_df.head()

,acc_state,state_lat,state_lng
0,WI,43.784440,-88.787868
1,WV,38.597626,-80.454903
2,OH,40.417287,-82.907123
3,WA,47.751074,-120.740139
4,CA,36.778261,-119.417932


In [104]:
df = pd.merge(df, city_df, how="outer", on="acc_city")
df = pd.merge(df, state_df, how="outer", on="acc_state")

In [105]:
df.drop_duplicates(subset="acc_id", inplace=True)
df.head()

,acc_id,acc_date,acc_state,acc_city,fix_port,source,bus_type,industry_sector,device_category,device_type,...,report,category,mechanical,op_error,employee,notes,city_lat,city_lng,state_lat,state_lng
0,1005755,9/12/2009,WI,Green Bay,F,"Wisconsin Dept. of Commerce, Safety and Buildi...",Amusement park,recreation,play equipment,Slide,...,NaN,Impact: hit something in participatory attraction,NaN,NaN,NaN,NaN,44.5133,-88.0133,43.78444,-88.787868
1,1005750,7/11/2008,WI,Green Bay,F,"Wisconsin Dept. of Commerce, Safety and Buildi...",Amusement park,amusement ride,spinning,Roundabout,...,NaN,Entrapment or pinch-point,NaN,NaN,NaN,NaN,44.5133,-88.0133,43.78444,-88.787868
2,916745,9/2/2006,WI,Green Bay,F,"Wisconsin Dept. of Commerce, Safety and Buildi...",Amusement park,amusement ride,spinning,Rotor/gravitron,...,NaN,Load/Unload: scrape or stumble,NaN,NaN,NaN,NaN,44.5133,-88.0133,43.78444,-88.787868
3,916746,8/29/2006,WI,Green Bay,F,"Wisconsin Dept. of Commerce, Safety and Buildi...",Amusement park,recreation,play equipment,Slide,...,NaN,Burn (includes friction burn),NaN,NaN,NaN,NaN,44.5133,-88.0133,43.78444,-88.787868
4,916747,8/23/2006,WI,Green Bay,F,"Wisconsin Dept. of Commerce, Safety and Buildi...",Amusement park,amusement ride,spinning,Rotor/gravitron,...,NaN,Load/Unload: scrape or stumble,NaN,NaN,NaN,NaN,44.5133,-88.0133,43.78444,-88.787868


In [106]:
# Create Database Connection
# ----------------------------------
# Creates a connection to our DB
engine = create_engine("sqlite:///../../Resources/park_accidents.sqlite", encoding="ANSI")
conn = engine.connect()

In [107]:
df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()

{'results': [{'address_components': [{'long_name': 'Fairfield', 'short_name': 'Fairfield', 'types': ['locality', 'political']}, {'long_name': 'Essex County', 'short_name': 'Essex County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'New Jersey', 'short_name': 'NJ', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}], 'formatted_address': 'Fairfield, NJ, USA', 'geometry': {'bounds': {'northeast': {'lat': 40.90888990000001, 'lng': -74.2544759}, 'southwest': {'lat': 40.8446669, 'lng': -74.340328}}, 'location': {'lat': 40.8837406, 'lng': -74.3059959}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 40.90888990000001, 'lng': -74.2544759}, 'southwest': {'lat': 40.8446669, 'lng': -74.340328}}}, 'place_id': 'ChIJ_cHFXtgAw4kR-nWBnEiqjwE', 'types': ['locality', 'political']}], 'status': 'OK'}
